### Goal

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import pandas as pd

# to prevent figure 
# to show in the notebook
import matplotlib
matplotlib.use('Agg')

import matplotlib.pyplot as plt

import plotting as plotting

### TODO: update this

In [3]:
date = "24-12-12(pot_label_prob)"
y_none = True
pathfile = f"Pulling data/{date}/single/"

max_100 = True

### From here; automatic run

In [4]:
root = f"/Users/michaelmoret/Library/CloudStorage/GoogleDrive-michael@externa.bio/.shortcut-targets-by-id/1BdUNsBjDh5Gee_76jCiKB1C_CwG0ercP/{pathfile}"

MODE = "single"
dev = False

In [5]:
if not root.endswith("/"):
    root += "/"

sep_category = "$" # separates categories like date, experiments, etc
sep_count = "%" # separated the number of hairs by experiment

In [6]:
all_dfs = []
all_fns = []

verbose = False
for file in os.listdir(root):
    if file.endswith(".txt"):
        print("\n*****************************")
        print(file)
        df = plotting.get_df_from_file(root + file,
                                       skip=10)
        if verbose:
            print("df length after skipping: ", len(df))
        # clean the df
        if MODE in file:
            df = plotting.clean_single(df, max_100=max_100)
            assert "single" in file
        else:
            raise ValueError(f"Not a {MODE} experiment")
        if verbose:
            print("df length after cleaning: ", len(df))
        
        splitted_name = file.split(sep_category)
        # get the experiments; i.e. not the date
        # not the single or triple etc
        splitted_name = splitted_name[2:-1]
        print("splitted_name: ", splitted_name)
        # add the experiment name in the dataframe
        all_names = []
        for entries in splitted_name:
            times_name = entries.split(sep_count)
            _times = int(times_name[0])
            _name = times_name[1]
            all_names += [_name] * _times
        print(f"len df: {len(df)}, len names: {len(all_names)}")
        df["Name"] = all_names
        # remove nan
        df = df.dropna()
        all_dfs.append(df)
        all_fns.append(file.replace(".txt", ""))


*****************************
20241212$IHNY$25%phosctrl$25% KPMI NPYS 0.01$25%KPMI NPYS 0.03$25% KPMI NPYS 0.05$single.txt
Missing records: []
splitted_name:  ['25%phosctrl', '25% KPMI NPYS 0.01', '25%KPMI NPYS 0.03', '25% KPMI NPYS 0.05']
len df: 100, len names: 100

*****************************
20241212$IHNY$25%phosctrl$25% KPM2 NPYS 0.01$25%KPM2 NPYS 0.03$25% KPM2 NPYS 0.05$single.txt
Missing records: []
splitted_name:  ['25%phosctrl', '25% KPM2 NPYS 0.01', '25%KPM2 NPYS 0.03', '25% KPM2 NPYS 0.05']
len df: 100, len names: 100

*****************************
20241212$IHNY$25%AActrl$25% KPM2 NPYS 0.01$25%KPM2 NPYS 0.03$25% KPM2 NPYS 0.05$ single.txt
Missing records: []
splitted_name:  ['25%AActrl', '25% KPM2 NPYS 0.01', '25%KPM2 NPYS 0.03', '25% KPM2 NPYS 0.05']
len df: 100, len names: 100

*****************************
20241212$IHNY$25%AActrl$25% KPMI NPYS 0.01$25%KPMI NPYS 0.03$25% KPMI NPYS 0.05$ single.txt
Missing records: []
splitted_name:  ['25%AActrl', '25% KPMI NPYS 0.01', '

In [7]:
def master_plots(df, header, ymin, ymax, savepath, show=False, verbose=False):
    fig, ax, removed = plotting.create_boxplot(df, header, ymin, ymax)
    #save removed data
    removed.to_csv(f"{savepath}{header}_removed.csv", index=True)
    if verbose:
        print(f"{len(removed)} removed data points for {header}")
    # plot
    fig.savefig(f"{savepath}{header}.png")
    if show: 
        plt.show()
        plt.close()

In [8]:
len(all_dfs), len(all_fns)

(4, 4)

In [9]:
all_fns

['20241212$IHNY$25%phosctrl$25% KPMI NPYS 0.01$25%KPMI NPYS 0.03$25% KPMI NPYS 0.05$single',
 '20241212$IHNY$25%phosctrl$25% KPM2 NPYS 0.01$25%KPM2 NPYS 0.03$25% KPM2 NPYS 0.05$single',
 '20241212$IHNY$25%AActrl$25% KPM2 NPYS 0.01$25%KPM2 NPYS 0.03$25% KPM2 NPYS 0.05$ single',
 '20241212$IHNY$25%AActrl$25% KPMI NPYS 0.01$25%KPMI NPYS 0.03$25% KPMI NPYS 0.05$ single']

In [10]:
for df, name in zip(all_dfs, all_fns):
    print("\n************************************")
    print(f"{name}\n")
    print(f"df length: {len(df)}")
    try:
        savepath = f"{root}{name}/"
        if dev:
            savepath = f"{root}dev/"
        os.makedirs(savepath, exist_ok=True)
        
        try:
            header = 'TENSILE_STRENGTH'
            if y_none:
                ymin = None
                ymax = None
            else:
                ymin = 90
                ymax = 325
            master_plots(df, header, ymin, ymax, savepath)
        except Exception as e:
            print(f"ERROR with {header}")
            print("Error:", e)

        try:
            header = 'BREAK_STRAIN(*)(#)'
            if y_none:
                ymin = None
                ymax = None
            else:
                ymin = 0
                ymax = 100
            master_plots(df, header, ymin, ymax, savepath)
        except Exception as e:
            print(f"ERROR with {header}")
            print("Error:", e)

        try:
            header = 'BREAK_LOAD'
            if y_none:
                ymin = None
                ymax = None
            else:
                ymin = 0
                ymax = 2.0
            master_plots(df, header, ymin, ymax, savepath)
        except Exception as e:
            print(f"ERROR with {header}")
            print("Error:", e)
        
        try:
            header = 'EMOD(*)(#)'
            if y_none:
                ymin = None
                ymax = None
            else:
                ymin = 2.5
                ymax = 8.0
            master_plots(df, header, ymin, ymax, savepath)
        except Exception as e:
            print(f"ERROR with {header}")
            print("Error:", e)

        # scatter plots
        y_col = 'TENSILE_STRENGTH'
        x_col = 'MEAN AREA'
        savedir = f"{savepath}correlation_plot/"
        plt = plotting.create_scatter_plot(df, x_col, y_col, savedir)

        y_col = 'EMOD(*)(#)'
        x_col = 'MEAN AREA'
        savedir = f"{savepath}correlation_plot/"
        plt = plotting.create_scatter_plot(df, x_col, y_col, savedir)

        y_col = 'BREAK_LOAD'
        x_col = 'MEAN AREA'
        savedir = f"{savepath}correlation_plot/"
        plt = plotting.create_scatter_plot(df, x_col, y_col, savedir)

    except:
        print(f"\nERROR with {name}\n")
        continue

print("\n\nplotting DONE")


************************************
20241212$IHNY$25%phosctrl$25% KPMI NPYS 0.01$25%KPMI NPYS 0.03$25% KPMI NPYS 0.05$single

df length: 93

Creating boxplot for TENSILE_STRENGTH
Removed outliers for phosctrl : 16    97.79
Name: TENSILE_STRENGTH, dtype: float64
Removed outliers for  KPMI NPYS 0.01 : Series([], Name: TENSILE_STRENGTH, dtype: float64)
Removed outliers for KPMI NPYS 0.03 : 57    150.2
Name: TENSILE_STRENGTH, dtype: float64
Removed outliers for  KPMI NPYS 0.05 : Series([], Name: TENSILE_STRENGTH, dtype: float64)

Creating boxplot for BREAK_STRAIN(*)(#)
Removed outliers for phosctrl : 16    7.594
Name: BREAK_STRAIN(*)(#), dtype: float64
Removed outliers for  KPMI NPYS 0.01 : Series([], Name: BREAK_STRAIN(*)(#), dtype: float64)
Removed outliers for KPMI NPYS 0.03 : Series([], Name: BREAK_STRAIN(*)(#), dtype: float64)
Removed outliers for  KPMI NPYS 0.05 : 84    27.189
91    33.489
Name: BREAK_STRAIN(*)(#), dtype: float64

Creating boxplot for BREAK_LOAD
Removed outliers fo

/Users/michaelmoret/tinyverse/krap/notebooks_2/plotting.py:238: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.figure(figsize=figsize)



************************************
20241212$IHNY$25%AActrl$25% KPMI NPYS 0.01$25%KPMI NPYS 0.03$25% KPMI NPYS 0.05$ single

df length: 88

Creating boxplot for TENSILE_STRENGTH
Removed outliers for AActrl : Series([], Name: TENSILE_STRENGTH, dtype: float64)
Removed outliers for  KPMI NPYS 0.01 : 37    164.3
Name: TENSILE_STRENGTH, dtype: float64
Removed outliers for KPMI NPYS 0.03 : 52    234.7
73    154.4
Name: TENSILE_STRENGTH, dtype: float64
Removed outliers for  KPMI NPYS 0.05 : Series([], Name: TENSILE_STRENGTH, dtype: float64)
Stats. significance: AActrl and  KPMI NPYS 0.01: p = 0.0188

Creating boxplot for BREAK_STRAIN(*)(#)
Removed outliers for AActrl : Series([], Name: BREAK_STRAIN(*)(#), dtype: float64)
Removed outliers for  KPMI NPYS 0.01 : Series([], Name: BREAK_STRAIN(*)(#), dtype: float64)
Removed outliers for KPMI NPYS 0.03 : 73    36.844
Name: BREAK_STRAIN(*)(#), dtype: float64
Removed outliers for  KPMI NPYS 0.05 : Series([], Name: BREAK_STRAIN(*)(#), dtype: float64